In [1]:
import yfinance as yf
import pandas as pd
import warnings
import plotly.express as px

from optimisation_markowitz import MarkowitzOptimization
from optimisation_simm import SingleFactorModel, SingleFactorModelOptimization
from utils import Utils
from plotly.subplots import make_subplots

warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

In [2]:
start_date = "2023-04-01"
end_date = "2024-04-01"
annualized_factor = 252

We will use S&P500 as benchmark to evaluate the performance with Markowitz Optimization and Single Index Market Model. The return over the last year will be used as the market return in the formula.

In [3]:
# Get close data for S&P 500 Index
market_index_data = Utils.get_historical_data("^SPX", start_date, end_date)

# calculate simple return for market index
market_index_simple_return = Utils.calculate_simple_return(market_index_data)

# calculate daily market data returns.
# use that to calculate expected returns and variance
market_index_daily_returns = Utils.calculate_daily_returns(market_index_data)
market_index_data_expected_return = market_index_daily_returns.mean()
market_index_data_variance = market_index_daily_returns.var()

print(f"Market Index Simple Return from {start_date} - {end_date} in %: ", market_index_simple_return * 100)
print("Market Index Expected daily Return in %: ", market_index_data_expected_return * 100)
print("Market Index daily Variance in %: ", market_index_data_variance * 100)

[*********************100%%**********************]  1 of 1 completed
Market Index Simple Return from 2023-04-01 - 2024-04-01 in %:  27.393324206617265
Market Index Expected daily Return in %:  0.10028595199917367
Market Index daily Variance in %:  0.005254458483042907


For markowitz model, we will use 3-Month US Treasury Bill as risk-free rate. Since we are holding the portfolio for 1 year, we will take the last value as the risk-free rate.

In [4]:
risk_free_rate_df = yf.download("^IRX", start_date, end_date)
risk_free_rate = risk_free_rate_df['Adj Close'].iloc[-1] / 100
risk_free_rate

[*********************100%%**********************]  1 of 1 completed


0.05203000068664551

# Combined Sector ETFs

Following our analysis for Technology, HealthCare and Financials sector, we will combine the 2 highest performing stocks from each sector in an effort to create a more diverse portfolio. The following stocks are included:
1. VanEck SemiConductor (SMH) -> Tech
2. iShares Semiconductor (SOXX) -> Tech
3. iShares US Financials ETF (IYF) -> Financials
4. Vanguard Financials ETF (VFH)  -> Financials
5. SPDR S&P Biotech ETF (XBI) -> HealthCare
6. Healthcare Sector SPDT Fund (XLV) -> HealthCare 

In [5]:
# get the data
smh_data = Utils.get_historical_data("SMH", start_date, end_date)
soxx_data = Utils.get_historical_data("SOXX", start_date, end_date)
iyf_data = Utils.get_historical_data("IYF", start_date, end_date)
vfh_data = Utils.get_historical_data("VFH", start_date, end_date)
xbi_data = Utils.get_historical_data("XBI", start_date, end_date)
xlv_data = Utils.get_historical_data("XLV", start_date, end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [6]:

smh_simple_return = Utils.calculate_simple_return(smh_data)
soxx_simple_return = Utils.calculate_simple_return(soxx_data)
iyf_simple_return = Utils.calculate_simple_return(iyf_data)
vfh_simple_return = Utils.calculate_simple_return(vfh_data)
xbi_simple_return = Utils.calculate_simple_return(xbi_data)
xlv_simple_return = Utils.calculate_simple_return(xlv_data)

simple_returns_dict = {
    "SMH": smh_simple_return,
    "SOXX": soxx_simple_return,
    "IYF": iyf_simple_return,
    "VFH": vfh_simple_return,
    "XBI": xbi_simple_return,
    "XLV": xlv_simple_return
}

for ticker, simple_return in simple_returns_dict.items():
    print(f"{ticker} -> Simple Return from {start_date} - {end_date} in %: ", simple_return * 100)

SMH -> Simple Return from 2023-04-01 - 2024-04-01 in %:  72.98801691288277
SOXX -> Simple Return from 2023-04-01 - 2024-04-01 in %:  57.11974274878562
IYF -> Simple Return from 2023-04-01 - 2024-04-01 in %:  36.68568980169884
VFH -> Simple Return from 2023-04-01 - 2024-04-01 in %:  34.56978857828583
XBI -> Simple Return from 2023-04-01 - 2024-04-01 in %:  23.162156584619375
XLV -> Simple Return from 2023-04-01 - 2024-04-01 in %:  14.699930196487564


In [7]:
# plot the simple retruns using plotly for all stocks along with market index
fig = px.line(title="Simple Returns for Tech Stocks and Market Index")
fig.add_scatter(x=smh_data.index, y=smh_data['simple_return'], mode='lines', name='SMH')
fig.add_scatter(x=soxx_data.index, y=soxx_data['simple_return'], mode='lines', name='SOXX')
fig.add_scatter(x=iyf_data.index, y=iyf_data['simple_return'], mode='lines', name='IYF')
fig.add_scatter(x=vfh_data.index, y=vfh_data['simple_return'], mode='lines', name='VFH')
fig.add_scatter(x=xbi_data.index, y=xbi_data['simple_return'], mode='lines', name='XBI')
fig.add_scatter(x=xlv_data.index, y=xlv_data['simple_return'], mode='lines', name='XLV')

fig.add_scatter(x=market_index_data.index, y=market_index_data['simple_return'], mode='lines', name='Market Index')
fig.show()

On observing the simple returns, we see that both ETFs from Healthcare sector underperformed relative to the market.

In [8]:
smh_daily_returns = Utils.calculate_daily_returns(smh_data)
soxx_daily_returns = Utils.calculate_daily_returns(soxx_data)
iyf_daily_returns = Utils.calculate_daily_returns(iyf_data)
vfh_daily_returns = Utils.calculate_daily_returns(vfh_data)
xbi_daily_returns = Utils.calculate_daily_returns(xbi_data)
xlv_daily_returns = Utils.calculate_daily_returns(xlv_data)


daily_returns_df = pd.DataFrame([smh_daily_returns, soxx_daily_returns, iyf_daily_returns, vfh_daily_returns, xbi_daily_returns, xlv_daily_returns]).T
daily_returns_df.columns = ["SMH", "SOXX", "IYF", "VFH", "XBI", "XLV"]

In [9]:
cov_matrix = daily_returns_df.cov()
cov_matrix

,SMH,SOXX,IYF,VFH,XBI,XLV
SMH,0.000298,0.000294,0.000061,0.000066,0.000073,0.000018
SOXX,0.000294,0.000304,0.000066,0.000071,0.000092,0.000019
IYF,0.000061,0.000066,0.000078,0.000077,0.000070,0.000028
VFH,0.000066,0.000071,0.000077,0.000078,0.000075,0.000029
XBI,0.000073,0.000092,0.000070,0.000075,0.000325,0.000055
XLV,0.000018,0.000019,0.000028,0.000029,0.000055,0.000044


In [10]:
corr_matrix = daily_returns_df.corr()
corr_matrix

,SMH,SOXX,IYF,VFH,XBI,XLV
SMH,1.000000,0.975606,0.402219,0.432813,0.233002,0.156475
SOXX,0.975606,1.000000,0.427095,0.458887,0.292228,0.162786
IYF,0.402219,0.427095,1.000000,0.986827,0.437255,0.475299
VFH,0.432813,0.458887,0.986827,1.000000,0.472434,0.491188
XBI,0.233002,0.292228,0.437255,0.472434,1.000000,0.459904
XLV,0.156475,0.162786,0.475299,0.491188,0.459904,1.000000


As expected, we observe that correlation among ETFs in the same sector is high whereas the correlation between ETFs in different sectors is low

In [11]:
# standard deviations between daily returns
std_dev = daily_returns_df.std()
std_dev

SMH     0.017272
SOXX    0.017427
IYF     0.008816
VFH     0.008851
XBI     0.018034
XLV     0.006632
dtype: float64

In [12]:
# daily mean return
mean_daily_return = daily_returns_df.mean()
mean_daily_return

SMH     0.002359
SOXX    0.001974
IYF     0.001300
VFH     0.001237
XBI     0.001002
XLV     0.000575
dtype: float64

## Optimal tangency portfolio using Markowitz model 

In [13]:
# Run the Markowitz Portfolio Optimization
simple_returns_list = list(simple_returns_dict.values())
markowitz = MarkowitzOptimization(simple_returns_list, cov_matrix, risk_free_rate)


In [14]:
# find the weights of the portfolio with short shelling
markowitz_weights_with_ss = markowitz.find_tangency_portfolio_with_short_selling()

In [16]:
# create the wieghts dictionary
markowitz_weights_with_ss_dict = {
    "SMH": markowitz_weights_with_ss[0],
    "SOXX": markowitz_weights_with_ss[1],
    "IYF": markowitz_weights_with_ss[2],
    "VFH": markowitz_weights_with_ss[3],
    "XBI": markowitz_weights_with_ss[4],
    "XLV": markowitz_weights_with_ss[5]
}

for ticker, weight in markowitz_weights_with_ss_dict.items():
    print(f"{ticker} -> Markowitz Weights with Short Selling in %: ", weight * 100)

SMH -> Markowitz Weights with Short Selling in %:  287.8126710176881
SOXX -> Markowitz Weights with Short Selling in %:  -253.68315970435603
IYF -> Markowitz Weights with Short Selling in %:  347.0677862764897
VFH -> Markowitz Weights with Short Selling in %:  -281.12332505634197
XBI -> Markowitz Weights with Short Selling in %:  12.511202399147455
XLV -> Markowitz Weights with Short Selling in %:  -12.585174932627307


In [17]:
# find the weights of the portfolio without short shelling
markowitz_weights_without_ss = markowitz.find_tangency_portfolio_no_short_selling()

In [18]:
# weights dictionary
markowitz_weights_without_ss_dict = {
    "SMH": markowitz_weights_without_ss[0],
    "SOXX": markowitz_weights_without_ss[1],
    "IYF": markowitz_weights_without_ss[2],
    "VFH": markowitz_weights_without_ss[3],
    "XBI": markowitz_weights_without_ss[4],
    "XLV": markowitz_weights_without_ss[5]
}

for ticker, weight in markowitz_weights_without_ss_dict.items():
    print(f"{ticker} -> Markowitz Weights without Short Selling in %: ", weight * 100)

SMH -> Markowitz Weights without Short Selling in %:  38.93543584965207
SOXX -> Markowitz Weights without Short Selling in %:  2.6043415033967608e-11
IYF -> Markowitz Weights without Short Selling in %:  61.06456415032409
VFH -> Markowitz Weights without Short Selling in %:  0.0
XBI -> Markowitz Weights without Short Selling in %:  2.413949496062133e-12
XLV -> Markowitz Weights without Short Selling in %:  0.0


In [19]:
volatility_with_ss = Utils.calculate_markowitz_portfolio_risk(markowitz_weights_with_ss, cov_matrix)
print("Volatility with Short Selling in %: ", volatility_with_ss * 100)

Volatility with Short Selling in %:  1.405934732563817


In [20]:
portfolio_mean_return_with_ss = Utils.calculate_portfolio_mean_return(markowitz_weights_with_ss, mean_daily_return, annualized_factor)
print("Mean Portfolio Return with Short Selling in %: ", portfolio_mean_return_with_ss * 100)

Mean Portfolio Return with Short Selling in %:  4.5553192532236535


In [21]:
# calculate sharpe ratio
sharpe_ratio_with_ss = Utils.calculate_sharpe_ratio(portfolio_mean_return_with_ss, volatility_with_ss, risk_free_rate)
print("Sharpe Ratio with Short Selling: ", sharpe_ratio_with_ss)

Sharpe Ratio with Short Selling:  0.6053016488750858


In [22]:
volatility_without_ss = Utils.calculate_markowitz_portfolio_risk(markowitz_weights_without_ss, cov_matrix)
print("Volatility without Short Selling in %: ", volatility_without_ss * 100)

Volatility without Short Selling in %:  1.016499421429571


In [23]:
portfolio_mean_return_without_ss = Utils.calculate_portfolio_mean_return(markowitz_weights_without_ss, mean_daily_return, annualized_factor)
print("Mean Portfolio Return without Short Selling in %: ", portfolio_mean_return_without_ss * 100)

Mean Portfolio Return without Short Selling in %:  2.718149469971438


In [24]:
# calculate sharpe ratio
sharpe_ratio_without_ss = Utils.calculate_sharpe_ratio(portfolio_mean_return_without_ss, volatility_without_ss, risk_free_rate)
print("Sharpe Ratio without Short Selling: ", sharpe_ratio_without_ss)

Sharpe Ratio without Short Selling:  0.3270517059552197


We also observe that without weights constraint in optimization without short selling, SMH and IYF account for around 99% proportion in the portfolio. To promote diversification , we will add a constraint that allows a maximum of 25% in each ETF

In [31]:
# we are interested in finding optimal weights when an asset can hold 30% of the portfolio max
markowitz_weights_without_ss_pct_constraint = markowitz.find_tangency_portfolio_no_short_selling_and_max_weight(0.25)

In [32]:
markowitz_weights_dict_without_ss_pct_constraint = {
    "SMH": markowitz_weights_without_ss_pct_constraint[0],
    "SOXX": markowitz_weights_without_ss_pct_constraint[1],
    "IYF": markowitz_weights_without_ss_pct_constraint[2],
    "VFH": markowitz_weights_without_ss_pct_constraint[3],
    "XBI": markowitz_weights_without_ss_pct_constraint[4],
    "XLV": markowitz_weights_without_ss_pct_constraint[5]
}

for ticker, weight in markowitz_weights_dict_without_ss_pct_constraint.items():
    print(f"{ticker} -> Markowitz Weights without Short Selling and 30% constraint in %: ", weight * 100)

SMH -> Markowitz Weights without Short Selling and 30% constraint in %:  24.99999999999705
SOXX -> Markowitz Weights without Short Selling and 30% constraint in %:  1.4643841694805815e-11
IYF -> Markowitz Weights without Short Selling and 30% constraint in %:  24.999999999992326
VFH -> Markowitz Weights without Short Selling and 30% constraint in %:  24.999999999994547
XBI -> Markowitz Weights without Short Selling and 30% constraint in %:  5.255988624505217e-12
XLV -> Markowitz Weights without Short Selling and 30% constraint in %:  24.99999995343006


In [33]:
volatility_without_ss_with_constraint = Utils.calculate_markowitz_portfolio_risk(markowitz_weights_without_ss_pct_constraint, cov_matrix)
print("Volatility with Short Selling and Constraint in %: ", volatility_without_ss_with_constraint * 100)

Volatility with Short Selling and Constraint in %:  0.8125095210818407


In [34]:
portfolio_mean_return_with_ss_with_constraint = Utils.calculate_portfolio_mean_return(markowitz_weights_without_ss_pct_constraint, mean_daily_return, annualized_factor)
print("Mean Portfolio Return with Short Selling and Constraint in %: ", portfolio_mean_return_with_ss_with_constraint * 100)

Mean Portfolio Return with Short Selling and Constraint in %:  2.1712671853168763


In [35]:
# find the Sharpe Ratio
sharpe_no_ss_with_constraint = Utils.calculate_sharpe_ratio(portfolio_mean_return_with_ss_with_constraint, volatility_without_ss_with_constraint, risk_free_rate)
print("Sharpe Ratio with Short Selling and Constraint: ", sharpe_no_ss_with_constraint)

Sharpe Ratio with Short Selling and Constraint:  0.2611488845480232


## Single Index Market Model

We will now compare results from markowitz model with single index market model. We will use S&P500 Index as benchmark to evaluate the performance with Single Index Market Model.

From this point, we will run `SingleFactorModel` class over all ETFs calculating beta, alpha and tau. We will then calculate the expected return using the formula:

$$
u_i = \alpha_i + \beta_i \cdot u_m

In [36]:
def get_simm_params(asset_return: pd.DataFrame, market_return: pd.DataFrame):
  model = SingleFactorModel(asset_return, market_return)
  model.fit()
  
  params = model.get_params()
  alpha = params.get("alpha").get("value")
  beta = params.get("beta").get("value")
  tau = params.get("tau").get("value")
  
  return (alpha, beta, tau)
  

In [37]:
# run SIMM for params
smh_alpha, smh_beta, smh_tau = get_simm_params(smh_daily_returns, market_index_daily_returns)
soxx_alpha, soxx_beta, soxx_tau = get_simm_params(soxx_daily_returns, market_index_daily_returns)
iyf_alpha, iyf_beta, iyf_tau = get_simm_params(iyf_daily_returns, market_index_daily_returns)
vfh_alpha, vfh_beta, vfh_tau = get_simm_params(vfh_daily_returns, market_index_daily_returns)
xbi_alpha, xbi_beta, xbi_tau = get_simm_params(xbi_daily_returns, market_index_daily_returns)
xlv_alpha, xlv_beta, xlv_tau = get_simm_params(xlv_daily_returns, market_index_daily_returns)

In [38]:
# get expected returns using alpha, beta and expected market return
smh_expected_return = smh_alpha + (smh_beta * market_index_data_expected_return)
soxx_expected_return = soxx_alpha + (soxx_beta * market_index_data_expected_return)
iyf_expected_return = iyf_alpha + (iyf_beta * market_index_data_expected_return)
vfh_expected_return = vfh_alpha + (vfh_beta * market_index_data_expected_return)
xbi_expected_return = xbi_alpha + (xbi_beta * market_index_data_expected_return)
xlv_expected_return = xlv_alpha + (xlv_beta * market_index_data_expected_return)

In [39]:
# we can now pack the expected returns into a dictionary along with beta and tau
expected_returns_dict = {
    "SMH": smh_expected_return,
    "SOXX": soxx_expected_return,
    "IYF": iyf_expected_return,
    "VFH": vfh_expected_return,
    "XBI": xbi_expected_return,
    "XLV": xlv_expected_return
}

alpha_dict = {
    "SMH": smh_alpha,
    "SOXX": soxx_alpha,
    "IYF": iyf_alpha,
    "VFH": vfh_alpha,
    "XBI": xbi_alpha,
    "XLV": xlv_alpha
}

beta_dict = {
    "SMH": smh_beta,
    "SOXX": soxx_beta,
    "IYF": iyf_beta,
    "VFH": vfh_beta,
    "XBI": xbi_beta,
    "XLV": xlv_beta
}

tau_dict = {
    "SMH": smh_tau,
    "SOXX": soxx_tau,
    "IYF": iyf_tau,
    "VFH": vfh_tau,
    "XBI": xbi_tau,
    "XLV": xlv_tau
}

for i in range(len(expected_returns_dict.keys())):
  print(f"{ list(expected_returns_dict.keys())[i] } -> Expected Daily Return in %: ", list(expected_returns_dict.values())[i] * 100)
  print(f"{ list(alpha_dict.keys())[i] } -> Alpha: ", list(alpha_dict.values())[i])
  print(f"{ list(beta_dict.keys())[i] } -> Beta: ", list(beta_dict.values())[i])
  print(f"{ list(tau_dict.keys())[i] } -> Tau: ", list(tau_dict.values())[i])
  print()

SMH -> Expected Daily Return in %:  0.2359472140425166
SMH -> Alpha:  0.0005917930464843844
SMH -> Beta:  1.762638793073776
SMH -> Tau:  0.011645399970193673

SOXX -> Expected Daily Return in %:  0.19738976059496985
SOXX -> Alpha:  0.0002046337968699332
SOXX -> Beta:  1.764218989609176
SOXX -> Tau:  0.01186329753014371

IYF -> Expected Daily Return in %:  0.12996120898916422
IYF -> Alpha:  0.00040636617625488804
IYF -> Beta:  0.8906989422049005
IYF -> Tau:  0.006014969929953517

VFH -> Expected Daily Return in %:  0.12369350957780294
VFH -> Alpha:  0.00029383984099442173
VFH -> Beta:  0.9404061446127355
VFH -> Tau:  0.0056573623674854604

XBI -> Expected Daily Return in %:  0.10016452150430175
XBI -> Alpha:  -6.010831406428891e-05
XBI -> Beta:  1.0587260807137322
XBI -> Tau:  0.016352726289893156

XLV -> Expected Daily Return in %:  0.05750597247127691
XLV -> Alpha:  4.57383314627797e-05
XLV -> Beta:  0.5278121039867587
XLV -> Tau:  0.0054276894235327005



In [40]:
# run the SIMM optimization
expected_returns_list = list(expected_returns_dict.values())
beta_list = list(beta_dict.values())
tau_list = list(tau_dict.values())
alpha_list = list(alpha_dict.values())

simm_optimization = SingleFactorModelOptimization(expected_returns_list, beta_list, tau_list, market_index_data_variance, risk_free_rate)

In [41]:
# find the optimal weights when short selling is allowed
simm_weights_with_ss = simm_optimization.find_weights_short_selling_allowed()

In [42]:
simm_weights_with_ss_dict = {
    "SMH": simm_weights_with_ss[0],
    "SOXX": simm_weights_with_ss[1],
    "IYF": simm_weights_with_ss[2],
    "VFH": simm_weights_with_ss[3],
    "XBI": simm_weights_with_ss[4],
    "XLV": simm_weights_with_ss[5]
}

for ticker, weight in simm_weights_with_ss_dict.items():
    print(f"{ticker} -> SIMM Weights with Short Selling in %: ", weight * 100)

SMH -> SIMM Weights with Short Selling in %:  -10.199527749738838
SOXX -> SIMM Weights with Short Selling in %:  -9.681175702622097
IYF -> SIMM Weights with Short Selling in %:  26.265125931733014
VFH -> SIMM Weights with Short Selling in %:  25.781720623643416
XBI -> SIMM Weights with Short Selling in %:  1.9928458065059114
XLV -> SIMM Weights with Short Selling in %:  65.8410110904786


With weights found, we can find the expected return of portfolio and the risk using the formula:

$$
u_p = \alpha_p + \beta_p \cdot u_m

\newline

\sigma_p^2 = \beta_p^2 \cdot \sigma_m^2 + \tau_p^2

\newline

where:
\newline

\alpha_p = \sum_{i=1}^{n} x_i \cdot \alpha_i

\newline

\beta_p = \sum_{i=1}^{n} x_i \cdot \beta_i

\newline

\tau_p^2 = \sum_{i=1}^{n} x_i^2 \cdot \tau_i^2

In [43]:
simm_portfolio_risk_with_ss = Utils.calculate_factor_model_percentage_portfolio_risk(simm_weights_with_ss, market_index_data_variance, alpha_list, beta_list, tau_list)
print("Portfolio Risk with Short Selling in %: ", simm_portfolio_risk_with_ss * 100)

Portfolio Risk with Short Selling in %:  0.5751141120311977


In [44]:
simm_portfolio_mean_return_with_ss = Utils.calculate_portfolio_mean_return(simm_weights_with_ss, mean_daily_return, annualized_factor)
print("Mean Portfolio Return with Short Selling in %: ", simm_portfolio_mean_return_with_ss * 100)

Mean Portfolio Return with Short Selling in %:  0.9954630418905714


In [45]:
# find the sharpe ratio
simm_sharpe_ratio_with_ss = Utils.calculate_sharpe_ratio(simm_portfolio_mean_return_with_ss, simm_portfolio_risk_with_ss, risk_free_rate)
print("SIMM Sharpe Ratio with Short Selling: ", simm_sharpe_ratio_with_ss)

SIMM Sharpe Ratio with Short Selling:  0.0807897221433757


In [46]:
# find optimal weights when short selling is not allowed
simm_weights_no_ss = simm_optimization.find_weights_no_short_selling()

In [47]:
simm_weights_no_ss_dict = {
    "SMH": simm_weights_no_ss[0],
    "SOXX": simm_weights_no_ss[1],
    "IYF": simm_weights_no_ss[2],
    "VFH": simm_weights_no_ss[3],
    "XBI": simm_weights_no_ss[4],
    "XLV": simm_weights_no_ss[5]
}

for ticker, weight in simm_weights_no_ss_dict.items():
  print(f"{ticker} -> SIMM Weights without Short Selling in %: ", weight * 100)
    

SMH -> SIMM Weights without Short Selling in %:  12.817093052996523
SOXX -> SIMM Weights without Short Selling in %:  12.361654072477632
IYF -> SIMM Weights without Short Selling in %:  24.27645198439146
VFH -> SIMM Weights without Short Selling in %:  28.97410823173594
XBI -> SIMM Weights without Short Selling in %:  3.90416969052186
XLV -> SIMM Weights without Short Selling in %:  17.666522967876592


In [48]:
# we will now work to find the expected return and risk of the portfolio
simm_portfolio_risk_no_ss = Utils.calculate_factor_model_percentage_portfolio_risk(simm_weights_no_ss, market_index_data_variance, alpha_list, beta_list, tau_list)
print("Portfolio Risk without Short Selling in %: ", simm_portfolio_risk_no_ss * 100)

Portfolio Risk without Short Selling in %:  0.8389200860727308


In [49]:
simm_portfolio_mean_return_without_ss = Utils.calculate_portfolio_mean_return(simm_weights_no_ss, mean_daily_return, annualized_factor)
print("Mean Portfolio Return without Short Selling in %: ", simm_portfolio_mean_return_without_ss * 100)

Mean Portfolio Return without Short Selling in %:  2.1605391560507137


In [51]:
# find the sharpe ratio
simm_sharpe_ratio_no_ss = Utils.calculate_sharpe_ratio(simm_portfolio_mean_return_without_ss, simm_portfolio_risk_no_ss, risk_free_rate)
print("SIMM Sharpe Ratio without Short Selling: ", simm_sharpe_ratio_no_ss)

SIMM Sharpe Ratio without Short Selling:  0.25401096531547834
